In [1]:
import requests
import json
import pyarrow as pa

In [4]:


# Define the API URL
url = "http://127.0.0.1:5000/create"
schema = pa.schema([
    pa.field("disclaimer", pa.string()),
    pa.field("file_path", pa.string()),
    pa.field("n_transitions", pa.int32()),
    pa.field("success", pa.bool_()),  # Corrected boolean field
    pa.field("success_labeled_by", pa.string())
])


fields_dict = {
    "fields": [
        {"name": field.name, "type": str(field.type) if str(field.type) != 'bool' else 'bool_'} for field in schema
    ]
}

# Define the payload
data = {
    "dataset": "ucsd_pick_and_place_dataset_converted_externally_to_rlds",
    "uri": "/Users/haoweichung/Development/CMU/Spring2024/FogROS2-RTX/FogX-Store/_datasets/dataset_db",
    "fields" : json.dumps(fields_dict)
}

# Send the POST request
response = requests.post(url, json=data)

# Print the response
print(response.status_code)
print(response.json())


500
{'message': "HTTPConnectionPool(host='localhost', port=11632): Max retries exceeded with url: /dataset/ucsd_pick_and_place_dataset_converted_externally_to_rlds (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1621e8d10>: Failed to establish a new connection: [Errno 61] Connection refused'))"}


In [36]:
import requests

# Define the API URL
url = "http://127.0.0.1:5000/write"

# Define the payload
data = {
    "ds_path": "gs://gresearch/robotics/ucsd_pick_and_place_dataset_converted_externally_to_rlds/0.1.0",
    "dataset" : "ucsd_pick_and_place_dataset_converted_externally_to_rlds",
    "uri": "/Users/haoweichung/Development/CMU/Spring2024/FogROS2-RTX/FogX-Store/_datasets/dataset_db"
}

# Send the POST request
response = requests.post(url, json=data)

# Print the response
print(response.status_code)
print(response.json())


200
{'message': 'Dataset written successfully'}


In [9]:
import lancedb
uri = "/Users/haoweichung/Development/CMU/Spring2024/FogROS2-RTX/FogX-Store/_datasets/dataset_db"
db = lancedb.connect(uri)
tbl = db.open_table("ucsd_pick_and_place_dataset_converted_externally_to_rlds")

In [13]:
tbl.to_arrow()

pyarrow.Table
disclaimer: string
file_path: string
n_transitions: int32
success: bool
success_labeled_by: string
----
disclaimer: []
file_path: []
n_transitions: []
success: []
success_labeled_by: []

In [7]:
result = (
    tbl.search()
    .where("success = true", prefilter=True)
    .to_arrow()
)

In [8]:
result

pyarrow.Table
disclaimer: string
file_path: string
n_transitions: int32
success: bool
success_labeled_by: string
----
disclaimer: []
file_path: []
n_transitions: []
success: []
success_labeled_by: []